# [모듈 4.1] 훈련 스텝 개발 (SageMaker Training Step)
---

이 노트북은 세이지 메이커의 Training Job을 통해서 데이터 전처리를 합니다. 
- 일반적으로 크게 아래 4가지의 스텝으로 진행이 됩니다.
    - S3에 훈련 파일 준비
        - 이전 단계의 전처리시의 결과 파일을 사용 합니다.
    - 훈련 알고리즘을 준비 (세이지 메이커 내장 알고리즘 혹은 사용자 정의 알고리즘 사용)
        - 이 노트북에서는 사용자 정의 알고리즘을 코드로 기술 했습니다.
    - Training Job을 생성시에 아래와 같은 항목을 제공합니다.
        - Training Job을 실행할 EC2(예: ml.m4.2xlarge) 기술
        - Ec2에서 로딩할 다커 이미지의 이름 기술
            - 아래 그림의 ECR(Amazon Elastic Container Registry)에서 다운로드
        - S3 입력 파일 경로
        - 훈련 코드 경로
        - 훈련 결과로서 모델 아티펙트 S3 경로 (지정하지 않으면 디폴트를 사용 합니다.)
    - EC2에서 훈련 실행 하여 S3에 모델 아티펙트 저장
- 상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 훈련](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)

![sagemaker-training.png](img/sagemaker-training.png)

---
## 위와 같은 전처리를 개발하기 위해서는 아래 3개의 단계를 수행합니다.

### (1) 로컬 노트북 인스턴스에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
- 로컬 노트북 인스턴스의 다커 컨테이너로 실행하기 이전에, 쥬피터 노트북에서 훈련 코드를 실행할 수 있습니다. 이 노트북에서는 이 과정을 생략 했습니다.
- [알림] 로컬 모드 참고 자료
    - 로컬모드 설명하는 블로그 자료 --> [Use the Amazon SageMaker local mode to train on your notebook instance](https://aws.amazon.com/blogs/machine-learning/use-the-amazon-sagemaker-local-mode-to-train-on-your-notebook-instance/)
    - TF, Pytorch, SKLean, SKLearn Processing JOb에 대한 로컬 모드 샘플 --> [Amazon SageMaker Local Mode Examples](https://github.com/aws-samples/amazon-sagemaker-local-mode)
    - Python SDK -->  [로컬모드 Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)


### (2) 세이지 메이커 호스트 모드로 (**로컬 노트북 인스턴스에서 실행이 되는 것이 아님**) 다커 컨테이너를 통해서 훈련 코드 실행      
- 위의 (1) 단계에서 훈련 코드의 로직 확인이 되었기에, 실제 세이지 메이커의 호스트 모드로 다커 컨테이너를 통해 훈련 코드를 수행 합니다.
    
### (3) SageMaker Pipeline 에서 훈련을 수행합니다.
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](img/https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [ ]:
import boto3
import sagemaker
import pandas as pd
import os

#region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

# region = sagemaker.Session().boto_region_name
# print("Using AWS Region: {}".format(region))

%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

## 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [127]:
! aws s3 ls {train_preproc_dir_artifact} --recursive

2021-06-27 07:19:36     682602 sklearn-fraud-process-2021-06-27-07-15-18-694/output/train/train.csv


In [128]:
train_preproc_dir_artifact_file = os.path.join(train_preproc_dir_artifact,'train.csv')
train_prep_df = pd.read_csv(train_preproc_dir_artifact_file)
train_prep_df

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


### 기본 훈련 변수 및 하이퍼파라미터 설정

In [108]:
def get_pos_scale_weight(df, label):
    '''
    1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
    예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
    호출:
        class_weight = get_pos_scale_weight(train_prep_df, label='fraud')
    '''
    fraud_sum = df[df[label] == 1].shape[0]
    non_fraud_sum = df[df[label] == 0].shape[0]
    class_weight = int(non_fraud_sum / fraud_sum)
    print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
    return class_weight
    
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')

fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


In [109]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = project_prefix

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


## 2. 훈련 스크립트 확인

전처리 코드는 크게 아래와 같이 구성 되어 있습니다.
- 커맨드 인자로 전달된 변수 내용 확인
- 훈련 데이터를 로딩 합니다.
- xgboost의 cross-validation(cv) 로 훈련 합니다.
- 훈련 성능을 나타내는 지표를 저장합니다.
- 훈련이 모델 아티펙트를 저장 합니다.
    - [알림] 일반적으로 xgboost의 알고리즘의 큰 변경이 없으면, 세이지 메이커 내장 xgboost 알고리즘을 사용합니다. 여기서는 훈련 코드를 사용자가 정의해서 사용할 수 있는 예시를 위하여 따로 훈련 코드를 만들었습니다.
---

In [110]:
# !pygmentize src/xgboost_starter_script.py

## 3. 훈련 코드 실행
---

### (1) 로컬 노트북 인스턴스에서 로컬 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행




In [111]:

xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = 'local',
    framework_version = "1.0-1")

In [112]:
xgb_estimator.fit(inputs = {'train': train_preproc_dir_artifact})


Creating 455tb6zx4m-algo-1-kdxcc ... 
Creating 455tb6zx4m-algo-1-kdxcc ... done
Attaching to 455tb6zx4m-algo-1-kdxcc
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker_xgboost_container.training:Invoking user training script.
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
455tb6zx4m-algo-1-kdxcc | Generating setup.py
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:Generating setup.cfg
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:Generating MANIFEST.in
455tb6zx4m-algo-1-kdxcc | INFO:sagemaker-containers:Installing module with the following command:
455tb6zx4m-algo-1-kdxcc | /miniconda3/bin/python3 -m pip install . 
455tb6zx4m-algo-1-kdxcc | Processing /opt/ml/code
455tb6zx4m-algo-1-kdxcc | Building wheels for collected

Failed to delete: /tmp/tmp59idt3hf/algo-1-kdxcc Please remove it manually.


===== Job Complete =====


### (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행

In [113]:
instance_type = 'ml.m5.xlarge'


xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = instance_type,
    framework_version = "1.0-1")

xgb_estimator.fit(inputs = {'train': train_preproc_dir_artifact}, wait=False)


### (3) SageMaker Pipeline에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [114]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)


input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [115]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.

In [116]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_dir_artifact,
            content_type="text/csv"
        ),
    },
)

### 모델 빌딩 파이프라인 정의

In [117]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
    ],
    steps=[step_train],
)

In [118]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [119]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': '9ff8e213-a9ee-482a-9960-aae4d9d36ee7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9ff8e213-a9ee-482a-9960-aae4d9d36ee7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Sun, 27 Jun 2021 07:20:23 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [120]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [121]:
execution.describe()

[]

In [122]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [123]:
execution.list_steps()

[{'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 6, 27, 7, 20, 25, 130000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 27, 7, 23, 50, 196000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-uny4eqq6iy0g-fraudscratchtrain-7bdiux6jba'}}}]

### 아티펙트 경로 추출
위의 훈련 스텝이 완료되면 실행해주세요

In [124]:
def get_train_artifact(execution, client, job_type,  kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    # print("response: ", response)
    proc_arn = response[0]['Metadata'][job_type]['Arn']
    train_job_name = proc_arn.split('/')[-1]
    # print("train_job_name: ", train_job_name)
    response = client.describe_training_job(TrainingJobName = train_job_name)
    # print("\nresponse: ", response)    
    train_model_artifact = response['ModelArtifacts']['S3ModelArtifacts']    
    
    return train_model_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


 train_model_artifact:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/training_jobs/pipelines-uny4eqq6iy0g-FraudScratchTrain-7bDiUX6Jba/output/model.tar.gz


In [125]:
image_uri = xgb_train.image_uri

In [126]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
